### Hanqing Zhao
#### CSE258 Midterm

In [504]:
import re
import json
import gzip
import math
from collections import defaultdict
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [505]:
# This will suppress any warnings, comment out if you'd like to preserve them
import warnings
warnings.filterwarnings("ignore")

In [370]:
# Check formatting of submissions
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [371]:
answers = {}

In [372]:
f = open("spoilers.json.gz", 'r')

In [373]:
dataset = []
for l in f:
    d = eval(l)
    dataset.append(d)

In [374]:
f.close()

In [375]:
# A few utility data structures
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

for d in dataset:
    u,i = d['user_id'],d['book_id']
    reviewsPerUser[u].append(d)
    reviewsPerItem[i].append(d)

# Sort reviews per user by timestamp
for u in reviewsPerUser:
    reviewsPerUser[u].sort(key=lambda x: x['timestamp'])
    
# Same for reviews per item
for i in reviewsPerItem:
    reviewsPerItem[i].sort(key=lambda x: x['timestamp'])

In [376]:
# E.g. reviews for this user are sorted from earliest to most recent
[d['timestamp'] for d in reviewsPerUser['b0d7e561ca59e313b728dc30a5b1862e']]

['2012-03-13',
 '2013-05-06',
 '2013-09-03',
 '2015-04-05',
 '2016-02-10',
 '2016-05-29']

In [377]:
def MSE(predictions, labels):
    return mean_squared_error(predictions, labels)

In [378]:
### 1a

In [379]:
y = []
ypred = []

for u in reviewsPerUser:
    if len(reviewsPerUser[u]) > 1:
        y.append(reviewsPerUser[u][-1]['rating'])
        total = sum(reviewsPerUser[u][i]['rating'] for i in range(len(reviewsPerUser[u])-1))
        average = total/(len(reviewsPerUser[u])-1)
        ypred.append(average)


In [380]:
answers['Q1a'] = MSE(y,ypred)
print(answers['Q1a'])

1.9704162943957526


In [381]:
assertFloat(answers['Q1a'])

In [382]:
### 1b

In [383]:
y_ = []
ypred_ = []

for i in reviewsPerItem:
    if len(reviewsPerItem[i]) > 1:
        y_.append(reviewsPerItem[i][-1]['rating'])
        total = sum(reviewsPerItem[i][x]['rating'] for x in range(len(reviewsPerItem[i][:-1])))
        average = total/len(reviewsPerItem[i][:-1])
        ypred_.append(average)


In [384]:
answers['Q1b'] = MSE(y_,ypred_)
print(answers['Q1b'])

2.05196610339507


In [385]:
assertFloat(answers['Q1b'])

In [386]:
### 2

In [387]:
def mean_N_ratings(N, data):
    y = []
    ypred = []

    for u in data:
        if len(data[u]) > N+1:
            y.append(data[u][-1]['rating'])
            total = sum(data[u][i]['rating'] for i in range(len(data[u][-(N+1):-1])))
            average = total/(len(reviewsPerUser[u][-(N+1):-1]))
            ypred.append(average)
        elif len(data[u]) <= N+1 and len(data[u]) > 1:
            y.append(data[u][-1]['rating'])
            total = sum(data[u][i]['rating'] for i in range(len(data[u][:-1])))
            average = total/(len(reviewsPerUser[u][:-1]))
            ypred.append(average)
    return y, ypred


In [388]:
answers['Q2'] = []

for N in [1,2,3]:
    y,ypred = mean_N_ratings(N,reviewsPerUser)
    answers['Q2'].append(MSE(y,ypred))

In [389]:
print(answers['Q2'])

[2.6814096499526965, 2.187263481551561, 2.0563439503836856]


In [390]:
assertFloatList(answers['Q2'], 3)

In [391]:
### 3a

In [392]:
def feature3(N, u): # For a user u and a window size of N
    output = [1]
    ratings = []
    for d in reviewsPerUser[u][-(N+1):-1]:
        ratings.append(d['rating'])
    output = output+ratings[::-1]
    return output

In [393]:
answers['Q3a'] = [feature3(2,dataset[0]['user_id']), feature3(3,dataset[0]['user_id'])]

In [394]:
answers['Q3a']

[[1, 4, 4], [1, 4, 4, 4]]

In [395]:
assert len(answers['Q3a']) == 2
assert len(answers['Q3a'][0]) == 3
assert len(answers['Q3a'][1]) == 4

In [396]:
### 3b

In [403]:
answers['Q3b'] = []

for N in [1,2,3]:
    X = []
    Y = []
    for u in reviewsPerUser:
        if len(reviewsPerUser[u]) >= N+1:
            X.append(feature3(N, u))
            Y.append(reviewsPerUser[u][-1]['rating'])
        
    theta, residuals, rank, s = np.linalg.lstsq(X,Y)
    ypred = np.dot(X,theta)
    mse = MSE(Y,ypred)
    answers['Q3b'].append(mse)

In [404]:
answers['Q3b']

[1.5608319121482543, 1.5409512373315701, 1.5396484853948416]

In [405]:
assertFloatList(answers['Q3b'], 3)

In [406]:
### 4a

In [407]:
globalAverage = [d['rating'] for d in dataset]
globalAverage = sum(globalAverage) / len(globalAverage)

In [410]:
def featureMeanValue(N, u): # For a user u and a window size of N
    output = [1]
    ratings = []
    #append global average
    if len(reviewsPerUser[u]) <= 1:
        for i in range(N):
            output.append(globalAverage)
    #append list's average
    elif len(reviewsPerUser[u]) < N+1:
        for d in reviewsPerUser[u][:-1]:
            ratings.append(d['rating'])
        output = output+ratings[::-1]
        ave = sum(ratings)/len(ratings)
        for i in range(len(reviewsPerUser[u])-1,N):
            output.append(ave)
    #append no average
    else:
        for d in reviewsPerUser[u][-(N+1):-1]:
            ratings.append(d['rating'])
        output = output+ratings[::-1]
    return output

In [411]:
def featureMissingValue(N, u):
    output = [1]
    ratings = []
    #append 1，0
    if len(reviewsPerUser[u]) < N+1:
        for d in reviewsPerUser[u][:-1]:
            ratings.append(d['rating'])
            ratings.append(0)
        output = output+ratings[::-1]
        for i in range(len(reviewsPerUser[u])-1,N):
            output += [1,0]
    #append only 0,x
    else:
        for d in reviewsPerUser[u][-(N+1):-1]:
            ratings.append(d['rating'])
            ratings.append(0)
        output = output+ratings[::-1]
    return output

In [412]:
answers['Q4a'] = [featureMeanValue(10, dataset[0]['user_id']), featureMissingValue(10, dataset[0]['user_id'])]

In [416]:
assert len(answers['Q4a']) == 2
assert len(answers['Q4a'][0]) == 11
assert len(answers['Q4a'][1]) == 21

In [414]:
### 4b

In [417]:
answers['Q4b'] = []

for featFunc in [featureMeanValue, featureMissingValue]:
    X = []
    Y = []
    for u in reviewsPerUser:
        X.append(featFunc(10, u))
        Y.append(reviewsPerUser[u][-1]['rating'])
        
    theta, residuals, rank, s = np.linalg.lstsq(X,Y)
    ypred = np.dot(X,theta)
    mse = MSE(Y,ypred)

    answers['Q4b'].append(mse)

In [419]:
answers['Q4b']

[1.5480131297119455, 1.5354064599216093]

In [420]:
assertFloatList(answers["Q4b"], 2)

In [421]:
### 5

In [496]:
def feature5(sentence):
    output = [1]
    output.append(len(sentence))
    output.append(sentence.count('!'))
    output.append(len(re.findall(r'[A-Z]',sentence)))
    return output

In [506]:
y = []
X = []

for d in dataset:
    for spoiler,sentence in d['review_sentences']:
        X.append(feature5(sentence))
        y.append(spoiler)

In [507]:
model = linear_model.LogisticRegression(fit_intercept=True, class_weight='balanced', C=1)
model.fit(X,y)

LogisticRegression(C=1, class_weight='balanced')

In [542]:
y_pred = model.predict(X)

In [543]:
def get_BER(predictions, y):
    TP_ = np.logical_and(predictions, y)
    FP_ = np.logical_and(predictions, np.logical_not(y))
    TN_ = np.logical_and(np.logical_not(predictions), np.logical_not(y))
    FN_ = np.logical_and(np.logical_not(predictions), y)
    
    TP = sum(TP_)
    FP = sum(FP_)
    TN = sum(TN_)
    FN = sum(FN_)
    
    BER = 1 - 0.5*(TP / (TP + FN) + TN / (TN + FP))
    return TP,TN,FP,FN,BER

In [544]:
TP, TN, FP, FN, BER = get_BER(y_pred, y)

In [545]:
answers['Q5a'] = X[0]

In [546]:
answers['Q5b'] = [TP, TN, FP, FN, BER]

In [547]:
assert len(answers['Q5a']) == 4
assertFloatList(answers['Q5b'], 5)

In [548]:
### 6

In [556]:
def feature6(review):
    output = []
    for i in range(5):
        #get the spoiler labels
        output.append(review['review_sentences'][i][0])
    output += feature5(review['review_sentences'][5][1])
    return output

In [557]:
y = []
X = []

for d in dataset:
    sentences = d['review_sentences']
    if len(sentences) >= 6:
        X.append(feature6(d))
        y.append(sentences[5][0])


In [560]:
model = linear_model.LogisticRegression(fit_intercept=True, class_weight='balanced', C=1)
model.fit(X,y)
y_pred = model.predict(X)

TP, TN, FP, FN, BER = get_BER(y_pred, y)

In [561]:
answers['Q6a'] = X[0]

In [562]:
answers['Q6b'] = BER

In [563]:
assert len(answers['Q6a']) == 9
assertFloat(answers['Q6b'])

In [564]:
### 7

In [565]:
# 50/25/25% train/valid/test split
Xtrain, Xvalid, Xtest = X[:len(X)//2], X[len(X)//2:(3*len(X))//4], X[(3*len(X))//4:]
ytrain, yvalid, ytest = y[:len(X)//2], y[len(X)//2:(3*len(X))//4], y[(3*len(X))//4:]

In [568]:
def BER(predictions, y):
    TP_ = np.logical_and(predictions, y)
    FP_ = np.logical_and(predictions, np.logical_not(y))
    TN_ = np.logical_and(np.logical_not(predictions), np.logical_not(y))
    FN_ = np.logical_and(np.logical_not(predictions), y)
    
    TP = sum(TP_)
    FP = sum(FP_)
    TN = sum(TN_)
    FN = sum(FN_)
    
    BER = 1 - 0.5*(TP / (TP + FN) + TN / (TN + FP))
    return BER

In [570]:
bers = []
bestC = 0
minBer = 10000
for c in [0.01, 0.1, 1, 10, 100]:
    mod = linear_model.LogisticRegression(C=c, class_weight='balanced')
    mod.fit(Xtrain,ytrain)
    pred_valid = mod.predict(Xvalid)
    berValid = BER(pred_valid, yvalid)
    bers.append(berValid)
    if berValid < minBer:
        minBer = berValid
        bestC = c

In [571]:
#Best c for test
mod_best = linear_model.LogisticRegression(C=bestC, class_weight='balanced')
mod_best.fit(Xtrain,ytrain)

pred_test = mod_best.predict(Xtest)
ber = BER(pred_test, ytest)

In [572]:
answers['Q7'] = bers + [bestC] + [ber]

In [573]:
assertFloatList(answers['Q7'], 7)

In [574]:
### 8

In [470]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [471]:
# 75/25% train/test split
dataTrain = dataset[:15000]
dataTest = dataset[15000:]

In [508]:
# A few utilities

itemAverages = defaultdict(list)
userAverages = defaultdict(list)
ratingMean = []

for d in dataTrain:
    itemAverages[d['book_id']].append(d['rating'])
    ratingMean.append(d['rating'])
    userAverages[d['user_id']].append(d['rating'])

for i in itemAverages:
    itemAverages[i] = sum(itemAverages[i]) / len(itemAverages[i])

for u in userAverages:
    userAverages[u] = sum(userAverages[u]) / len(userAverages[u])

ratingMean = sum(ratingMean) / len(ratingMean)

In [509]:
reviewsPerUser = defaultdict(list)
usersPerItem = defaultdict(set)
ratingDict = defaultdict(list)
for d in dataTrain:
    u,i = d['user_id'], d['book_id']
    reviewsPerUser[u].append(d)
    usersPerItem[i].add(u)
    ratingDict[(u,i)] = d['rating']

In [510]:
# From my HW2 solution, welcome to reuse
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        if item in itemAverages:
            return itemAverages[item]
        else:
            return ratingMean

In [511]:
predictions = [predictRating(d['user_id'], d['book_id']) for d in dataTest]
labels = [d['rating'] for d in dataTest]


In [512]:
answers["Q8"] = MSE(predictions, labels)
print(answers["Q8"])

1.816493441279133


In [513]:
assertFloat(answers["Q8"])

In [514]:
### 9

In [515]:
data_0 = []
data_1to5 = []
data_5 = []

def count_occur(dataTrain, book_id):
    occur = 0
    for d in dataTrain:
        if d['book_id'] == book_id:
            occur += 1
    return occur

In [516]:
for d in dataTest:
    count = count_occur(dataTrain, d['book_id'])
    if count == 0:
        data_0.append(d)
    elif count >=1 and count <= 5:
        data_1to5.append(d)
    elif count > 5:
        data_5.append(d)

In [517]:
pred0 = [predictRating(d['user_id'], d['book_id']) for d in data_0]
pred1to5 = [predictRating(d['user_id'], d['book_id']) for d in data_1to5]
pred5 = [predictRating(d['user_id'], d['book_id']) for d in data_5]

labels0 = [d['rating'] for d in data_0]
labels1to5 = [d['rating'] for d in data_1to5]
labels5 = [d['rating'] for d in data_5]


In [518]:
mse0 = MSE(pred0, labels0)
mse1to5 = MSE(pred1to5, labels1to5)
mse5 = MSE(pred5, labels5)

In [519]:
answers["Q9"] = [mse0, mse1to5, mse5]
print(answers["Q9"])

[1.7420124844444445, 2.0526818720058895, 1.4520632348645055]


In [520]:
assertFloatList(answers["Q9"], 3)

In [521]:
### 10

In [529]:
def predictRating2(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        if item in itemAverages:
            return itemAverages[item]
        else:
            if user in userAverages:
                return userAverages[user]
            else:
                return ratingMean

In [530]:
pred0_ = [predictRating2(d['user_id'], d['book_id']) for d in data_0]
labels0_ = [d['rating'] for d in data_0]

In [533]:
itsMSE = MSE(pred0_, labels0_)
print(itsMSE)

1.6696633366192306


In [576]:
answers["Q10"] = ("Since for the data that are not in the training set, we always return the global ratingMean because their similarities are always zero. I changed the else condition, which is when the similarities are 0, to return the average rating that a specific user will gave, to introduce more reasonable variances based on the user's rating histories and thus could reduce the MSE a little bit.", itsMSE)

In [577]:
assert type(answers["Q10"][0]) == str
assertFloat(answers["Q10"][1])

In [578]:
f = open("answers_midterm.txt", 'w')
f.write(str(answers) + '\n')
f.close()